In [120]:
import pymupdf

doc = pymupdf.open("./docs/trac-nghiem-ccna_compress.pdf")

In [121]:
DUMMY_QUESTION = {
    "question": "",
    "answers": [],
    "correct_answer": None,
}

In [122]:
import re
import json
from copy import deepcopy

questions = []
current_question = None

for page in doc:
    blocks = page.get_text("dict", flags=11)["blocks"]
    for b in blocks:  # iterate through the text blocks
        for l in b["lines"]:  # iterate through the text lines
            for s in l["spans"]:  # iterate through the text spans
                if s["text"].startswith("CÂU") or s["text"].startswith("Câu"):
                    if current_question is not None:
                        questions.append(current_question)
                    current_question = deepcopy(DUMMY_QUESTION)
                elif s["text"][:2] in ["a.", "b.", "c.", "d."]:
                    if s["color"] == 16711680:
                        current_question["correct_answer"] = len(
                            current_question["answers"]
                        )

                    current_question["answers"].append(s["text"][3:].strip())
                else:
                    current_question["question"] += s["text"]

if current_question is not None:
    questions.append(current_question)

# filter questions with no correct answer
questions = [q for q in questions if q["correct_answer"] is not None]

print(len(questions))

107


In [123]:
questions_1 = questions

In [124]:
from bs4 import BeautifulSoup

with open("./docs/Computer NetWork _ AI BLOG.html") as f:
    html_content = f.read()

soup = BeautifulSoup(html_content, "html.parser")
content = soup.find("article")

questions = []
current_question = None

# each children in content
for child in content.children:
    if child.name == "p":
        if child.text.startswith("Câu"):
            if current_question is not None:
                questions.append(current_question)
            current_question = deepcopy(DUMMY_QUESTION)
            current_question["question"] = re.sub(r"Câu \d+", "", child.text).strip()

            if current_question["question"].startswith("."):
                current_question["question"] = current_question["question"][1:].strip()

    elif child.name == "ul":

        # if classname is task-list
        if child.get("class") == ["task-list"]:
            for li in child.find_all("li"):
                checkbox = li.find("input")
                if checkbox is not None:
                    if checkbox.get("checked") is not None:
                        if current_question["correct_answer"] is None:
                            current_question["correct_answer"] = len(
                                current_question["answers"]
                            )
                        elif type(current_question["correct_answer"]) == int:
                            current_question["correct_answer"] = [
                                current_question["correct_answer"],
                                len(current_question["answers"]),
                            ]
                        else:
                            current_question["correct_answer"].append(
                                len(current_question["answers"])
                            )
                current_question["answers"].append(li.text)
        else:
            current_question["question"] += "\n" + child.text

    elif child.name == "blockquote":
        explanation = child.text.split("Giải thích:")[1].strip()

        if explanation != "None" and explanation != "":
            current_question["explanation"] = explanation

if current_question is not None:
    questions.append(current_question)

# filter questions with no correct answer
questions = [q for q in questions if q["correct_answer"] is not None]

questions_2 = questions

print(len(questions_2))

167


In [125]:
doc = pymupdf.open("./docs/mmt-quiz-bank.pdf")

questions = []
current_question = None
answer_set = set()

i = 0
for page in doc:
    blocks = page.get_text("dict", flags=11)["blocks"]
    for b in blocks:
        for l in b["lines"]:
            for s in l["spans"]:
                if s["text"].startswith("Câu"):
                    if current_question is not None:
                        questions.append(current_question)

                    answer_set = set()
                    current_question = deepcopy(DUMMY_QUESTION)
                    current_question["question"] = re.sub(
                        r"Câu \d+.", "", s["text"]
                    ).strip()
                elif current_question is not None:
                    if s["text"][:2] in [
                        "a.",
                        "b.",
                        "c.",
                        "d.",
                        "e.",
                        "f.",
                        "g.",
                    ] and not (s["text"][:2] in answer_set):
                        answer_set.add(s["text"][:2])

                        if "Bold" in s["font"]:
                            if current_question["correct_answer"] is None:
                                current_question["correct_answer"] = len(
                                    current_question["answers"]
                                )
                            elif type(current_question["correct_answer"]) == int:
                                current_question["correct_answer"] = [
                                    current_question["correct_answer"],
                                    len(current_question["answers"]),
                                ]
                            else:
                                current_question["correct_answer"].append(
                                    len(current_question["answers"])
                                )

                        current_question["answers"].append(s["text"][2:].strip())
                    elif "Gợi ý:" in s["text"]:
                        current_question["explanation"] = re.sub(
                            r"Gợi ý:", "", s["text"]
                        )
                    else:
                        if "explanation" in current_question:
                            current_question["explanation"] += "\n" + s["text"]
                        elif len(current_question["answers"]) > 0:
                            current_question["answers"][-1] += s["text"]
                        else:
                            current_question["question"] += s["text"]

if current_question is not None:
    questions.append(current_question)

# remove "Gợi ý:" in explanation
for q in questions:
    if "explanation" in q:
        q["explanation"] = re.sub(r"Gợi ý:", "", q["explanation"]).strip()

# filter questions with no correct answer
questions = [q for q in questions if q["correct_answer"] is not None]

print(len(questions))

150


In [126]:
questions_3 = questions

In [127]:
# total questions
print(len(questions_1) + len(questions_2) + len(questions_3))

424


In [128]:
with open("./data/cn.json", "w") as f:
    json.dump(
        [*questions_1, *questions_2, *questions_3], f, ensure_ascii=False, indent=4
    )